# Введение в нейронные сети

## Урок 5. Рекуррентные нейронные сети

### Практическое задание

1. Попробуйте изменить параметры нейронной сети, работающей с датасетом imdb, либо
нейронной сети, работающей airline-passengers (она прилагается вместе с датасетом к
уроку в виде отдельного скрипта) так, чтобы улучшить её точность. Приложите анализ.


2. Попробуйте изменить параметры нейронной сети, генерирующей текст таким образом,
чтобы добиться генерации как можно более осмысленного текста. Пришлите лучший
текст из получившихся и опишите предпринятые для его получения действия. Можно
использовать текст другого произведения.

### Решение

#### Импорт библиотек

In [2]:
import numpy as np
import pandas as pd

import tensorflow as tf

from tensorflow import keras

from tensorflow.keras.preprocessing import sequence

from tensorflow.keras.models import Sequential

from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import SimpleRNN
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import GRU

from tensorflow.keras.datasets import imdb

#### Глобальные настройки проекта

In [3]:
# Случайное зерно.
GLOBAL__RANDOM_STATE = 0

# Путь к тексту.
PATH__WEIGHTS_BASELINE = r'alice_in_wonderland.txt'

#### Задание 1 - датасет imdb

In [12]:
# Параметры нейросети.
max_features = 20000
max_len = 80
batch_size = 50

In [5]:
# Загрузка данных.
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=max_features)

# Проверка.
X_train.shape, y_train.shape, X_test.shape, y_test.shape

17464789/17464789 [==============================] - 2s 0us/step


((25000,), (25000,), (25000,), (25000,))

In [7]:
# Словарь слово-индекс.
word_index = imdb.get_word_index()

# Словарь индекс-слово.
inverted_word_index = dict((i, word) for (word, i) in word_index.items())

1641221/1641221 [==============================] - 0s 0us/step


In [23]:
# Усечение текстов.
X_train = sequence.pad_sequences(X_train, maxlen=maxlen)
X_test = sequence.pad_sequences(X_test, maxlen=maxlen)

##### Исходная модель из урока

In [26]:
# Настройка модели.
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

In [27]:
# Компиляция.
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [29]:
%%time

# Обучение.
model.fit(X_train,
          y_train,
          batch_size=batch_size,
          epochs=1,
          validation_data=(X_test, y_test))

500/500 [==============================] - 47s 91ms/step - loss: 0.4276 - accuracy: 0.7994 - val_loss: 0.3512 - val_accuracy: 0.8464
Wall time: 47.2 s


In [30]:
%%time

# Таблица метрик.
df_metrics = pd.DataFrame()

# Оценка модели.
df_metrics.loc['Baseline', 'Loss'], df_metrics.loc['Baseline', 'Accuracy'] = (
    model.evaluate(X_test, y_test, batch_size=batch_size)
)

# Вывод метрик.
df_metrics

500/500 [==============================] - 6s 11ms/step - loss: 0.3512 - accuracy: 0.8464
Wall time: 5.67 s


,Loss,Accuracy
Baseline,0.351244,0.84636


##### Улучшенная модель

In [83]:
# Настройка модели.
model = Sequential()
model.add(Embedding(max_features, 128))
# model.add(Embedding(max_features, 128 * 2))
# model.add(LSTM(128, dropout=0.1, recurrent_dropout=0.1))
# model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2, return_sequences=True))
model.add(LSTM(ваи128, dropout=0.2, recurrent_dropout=0.2))
# model.add(LSTM(128 * 2, dropout=0.2, recurrent_dropout=0.2))
# model.add(Dense(128, activation='sigmoid'))
model.add(Dense(1, activation='sigmoid'))

In [84]:
# Компиляция.
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [85]:
%%time

# Обучение.
model.fit(X_train,
          y_train,
          batch_size=batch_size,
          epochs=1,
#           epochs=3,
#           epochs=5,
          validation_data=(X_test, y_test))

500/500 [==============================] - 265s 527ms/step - loss: 0.4223 - accuracy: 0.8030 - val_loss: 0.3625 - val_accuracy: 0.8391
Wall time: 4min 25s


In [86]:
%%time

# Оценка модели.
df_metrics.loc['Improved', 'Loss'], df_metrics.loc['Improved', 'Accuracy'] = (
    model.evaluate(X_test, y_test, batch_size=batch_size)
)

# Вывод метрик.
df_metrics

500/500 [==============================] - 53s 106ms/step - loss: 0.3625 - accuracy: 0.8391
Wall time: 53.2 s


,Loss,Accuracy
Baseline,0.351244,0.84636
Improved,0.362505,0.83908


##### Выводы

Ни один из следующих методов не позволил добиться улучшения метрики качества <code>Accuracy</code>:
- увеличение количества эпох обучения модели;
- добавление полносвязных слоёв <code>Dense</code> или рекуррентных слоёв <code>LSTM</code>;
- увеличение количества нейронов в полносвязных и рекуррентных слоях;
- изменение функций активации и метода оптимизации;
- изменение доли обнуляемых весов при помощи настройки параметров <code>dropout</code> и <code>recurrent_dropout</code>.

Возможно, из данных был получен максимум информации для данной модели и для получения лучших результатов требуется применение более совершенных алгоритмов.

#### Задание 2 - генерация текста

In [87]:
# загрузка данных.
with open("alice_in_wonderland.txt", 'rb') as _in:
    lines = []
    for line in _in:
        line = line.strip().lower().decode("ascii", "ignore")
        if len(line) == 0:
            continue
        lines.append(line)
text = " ".join(lines)
chars = set(text)
nb_chars = len(chars)

In [91]:
# Словарь символ-индекс.
char2index = {c: i for i, c in enumerate(chars)}

# Словарь индекс-символ.
index2char = {i: c for i, c in enumerate(chars)}

##### Исходная реализация признаков и модели

In [105]:
# Генерация признаков, последовательность символов, и целевой переменной, следующего символа.
SEQLEN, STEP = 10, 1
input_chars, label_chars = [], []

for i in range(0, len(text) - SEQLEN, STEP):
    input_chars.append(text[i: i + SEQLEN])
    label_chars.append(text[i + SEQLEN])

In [106]:
# One-hot encoding признаков и целевой переменной.
X = np.zeros((len(input_chars), SEQLEN, nb_chars), dtype=np.bool_)
y = np.zeros((len(input_chars), nb_chars), dtype=np.bool_)

for i, input_char in enumerate(input_chars):
    for j, ch in enumerate(input_char):
        X[i, j, char2index[ch]] = 1
    y[i, char2index[label_chars[i]]] = 1

In [107]:
# Проверка: вывод размерностей признаков и целевой переменной.
X.shape, y.shape

((158773, 10, 55), (158773, 55))

In [108]:
# Параметры нейросети.
BATCH_SIZE, HIDDEN_SIZE = 128, 128
NUM_ITERATIONS = 10
NUM_EPOCHS_PER_ITERATION = 1
NUM_PREDS_PER_EPOCH = 100

In [109]:
model = Sequential()
model.add(GRU(HIDDEN_SIZE,
              return_sequences=False,
              input_shape=(SEQLEN, nb_chars),
              unroll=True))
model.add(Dense(nb_chars))
model.add(Activation("softmax"))
model.compile(loss="categorical_crossentropy", optimizer="rmsprop")

In [110]:
%%time

for iteration in range(NUM_ITERATIONS):
    # Обучение модели.
    print('')
    print('=' * 50)
    print(f'Итерация {iteration}')
    model.fit(X, y, batch_size=BATCH_SIZE, epochs=NUM_EPOCHS_PER_ITERATION)
    
    # Выбор случайной последовательности для проверки модели.
    test_idx = np.random.randint(len(input_chars))
    test_chars = input_chars[test_idx]
    
    # Вывод выбранной последовательности.
    print(f'Генерация из посева:\t{test_chars}')
#     print(f'[{test_chars}]', end="")
    print(test_chars, end="")
    
    # Прогноз.
    for i in range(NUM_PREDS_PER_EPOCH):
        
        # Ohe-hot encoding тестовой выборки.
        X_test = np.zeros((1, SEQLEN, nb_chars))
        for j, ch in enumerate(test_chars):
            X_test[0, j, char2index[ch]] = 1
        
        # Прогноз на тестовой выборке.
        pred = model.predict(X_test, verbose=0)[0]
        y_pred = index2char[np.argmax(pred)]
        
        # Вывод прогнозируемого символа.
        print(y_pred, end="")
        
        # Добавление спрогнозированного символа в тестовую выборку,
        # чтобы в следующий раз делать прогноз с учётом текущего спрогнозируемого символа.
        test_chars = test_chars[1:] + y_pred


Итерация 0
1241/1241 [==============================] - 17s 13ms/step - loss: 2.3184
Генерация из посева:	ed to list
ed to list ou the wand the mand the mand the mand the mand the mand the mand the mand the mand the mand the ma
Итерация 1
1241/1241 [==============================] - 15s 12ms/step - loss: 1.9182
Генерация из посева:	 us with o
 us with of the mack to the mack to the mack to the mack to the mack to the mack to the mack to the mack to th
Итерация 2
1241/1241 [==============================] - 15s 12ms/step - loss: 1.7512
Генерация из посева:	ll that. w
ll that. when the doresting the doresting the doresting the doresting the doresting the doresting the dorestin
Итерация 3
1241/1241 [==============================] - 15s 12ms/step - loss: 1.6423
Генерация из посева:	d the dorm
d the dormouse she was so she was so she was so she was so she was so she was so she was so she was so she was
Итерация 4
1241/1241 [==============================] - 15s 12ms/step - loss: 1.5624
Ге

##### Улучшенная модель - GRU и последовательность из 30 символов

In [118]:
# Генерация признаков, последовательность символов, и целевой переменной, следующего символа.
# SEQLEN, STEP = 10, 1
SEQLEN, STEP = 30, 1
input_chars, label_chars = [], []

for i in range(0, len(text) - SEQLEN, STEP):
    input_chars.append(text[i: i + SEQLEN])
    label_chars.append(text[i + SEQLEN])

In [119]:
# One-hot encoding признаков и целевой переменной.
X = np.zeros((len(input_chars), SEQLEN, nb_chars), dtype=np.bool_)
y = np.zeros((len(input_chars), nb_chars), dtype=np.bool_)

for i, input_char in enumerate(input_chars):
    for j, ch in enumerate(input_char):
        X[i, j, char2index[ch]] = 1
    y[i, char2index[label_chars[i]]] = 1

In [120]:
# Проверка: вывод размерностей признаков и целевой переменной.
X.shape, y.shape

((158753, 30, 55), (158753, 55))

In [121]:
# Параметры нейросети.
BATCH_SIZE, HIDDEN_SIZE = 128, 128
NUM_ITERATIONS = 10
NUM_EPOCHS_PER_ITERATION = 1
NUM_PREDS_PER_EPOCH = 100

In [122]:
model = Sequential()
model.add(GRU(HIDDEN_SIZE,
              return_sequences=False,
              input_shape=(SEQLEN, nb_chars),
              unroll=True))
model.add(Dense(nb_chars))
model.add(Activation("softmax"))
model.compile(loss="categorical_crossentropy", optimizer="rmsprop")

In [123]:
%%time

for iteration in range(NUM_ITERATIONS):
    # Обучение модели.
    print('')
    print('=' * 50)
    print(f'Итерация {iteration}')
    model.fit(X, y, batch_size=BATCH_SIZE, epochs=NUM_EPOCHS_PER_ITERATION)
    
    # Выбор случайной последовательности для проверки модели.
    test_idx = np.random.randint(len(input_chars))
    test_chars = input_chars[test_idx]
    
    # Вывод выбранной последовательности.
    print(f'Генерация из посева:\t{test_chars}')
#     print(f'[{test_chars}]', end="")
    print(test_chars, end="")
    
    # Прогноз.
    for i in range(NUM_PREDS_PER_EPOCH):
        
        # Ohe-hot encoding тестовой выборки.
        X_test = np.zeros((1, SEQLEN, nb_chars))
        for j, ch in enumerate(test_chars):
            X_test[0, j, char2index[ch]] = 1
        
        # Прогноз на тестовой выборке.
        pred = model.predict(X_test, verbose=0)[0]
        y_pred = index2char[np.argmax(pred)]
        
        # Вывод прогнозируемого символа.
        print(y_pred, end="")
        
        # Добавление спрогнозированного символа в тестовую выборку,
        # чтобы в следующий раз делать прогноз с учётом текущего спрогнозируемого символа.
        test_chars = test_chars[1:] + y_pred

    print('')


Итерация 0
1241/1241 [==============================] - 55s 41ms/step - loss: 2.2959
Генерация из посева:	 provide a secure and permanen
 provide a secure and permanen the said the said the said the said the said the said the said the said the said the said the said 

Итерация 1
1241/1241 [==============================] - 50s 40ms/step - loss: 1.9094
Генерация из посева:	ad through the doorway; and ev
ad through the doorway; and ever all the said the said the said the said the said the said the said the said the said the said the

Итерация 2
1241/1241 [==============================] - 50s 40ms/step - loss: 1.7459
Генерация из посева:	little door, so she went back 
little door, so she went back the could the could the could the could the could the could the could the could the could the could 

Итерация 3
1241/1241 [==============================] - 50s 40ms/step - loss: 1.6374
Генерация из посева:	se, that she did not like to g
se, that she did not like to get in a little said to th

##### Улучшенная модель - LSTM и последовательность из 30 символов

In [124]:
# Генерация признаков, последовательность символов, и целевой переменной, следующего символа.
# SEQLEN, STEP = 10, 1
SEQLEN, STEP = 30, 1
input_chars, label_chars = [], []

for i in range(0, len(text) - SEQLEN, STEP):
    input_chars.append(text[i: i + SEQLEN])
    label_chars.append(text[i + SEQLEN])

In [125]:
# One-hot encoding признаков и целевой переменной.
X = np.zeros((len(input_chars), SEQLEN, nb_chars), dtype=np.bool_)
y = np.zeros((len(input_chars), nb_chars), dtype=np.bool_)

for i, input_char in enumerate(input_chars):
    for j, ch in enumerate(input_char):
        X[i, j, char2index[ch]] = 1
    y[i, char2index[label_chars[i]]] = 1

In [126]:
# Проверка: вывод размерностей признаков и целевой переменной.
X.shape, y.shape

((158753, 30, 55), (158753, 55))

In [127]:
# Параметры нейросети.
BATCH_SIZE, HIDDEN_SIZE = 128, 128
NUM_ITERATIONS = 10
NUM_EPOCHS_PER_ITERATION = 1
NUM_PREDS_PER_EPOCH = 100

In [128]:
model = Sequential()
model.add(LSTM(HIDDEN_SIZE,
               return_sequences=False,
               input_shape=(SEQLEN, nb_chars),
               unroll=True))
model.add(Dense(nb_chars))
model.add(Activation("softmax"))
model.compile(loss="categorical_crossentropy", optimizer="rmsprop")

In [129]:
%%time

for iteration in range(NUM_ITERATIONS):
    # Обучение модели.
    print('')
    print('=' * 50)
    print(f'Итерация {iteration}')
    model.fit(X, y, batch_size=BATCH_SIZE, epochs=NUM_EPOCHS_PER_ITERATION)
    
    # Выбор случайной последовательности для проверки модели.
    test_idx = np.random.randint(len(input_chars))
    test_chars = input_chars[test_idx]
    
    # Вывод выбранной последовательности.
    print(f'Генерация из посева:\t{test_chars}')
#     print(f'[{test_chars}]', end="")
    print(test_chars, end="")
    
    # Прогноз.
    for i in range(NUM_PREDS_PER_EPOCH):
        
        # Ohe-hot encoding тестовой выборки.
        X_test = np.zeros((1, SEQLEN, nb_chars))
        for j, ch in enumerate(test_chars):
            X_test[0, j, char2index[ch]] = 1
        
        # Прогноз на тестовой выборке.
        pred = model.predict(X_test, verbose=0)[0]
        y_pred = index2char[np.argmax(pred)]
        
        # Вывод прогнозируемого символа.
        print(y_pred, end="")
        
        # Добавление спрогнозированного символа в тестовую выборку,
        # чтобы в следующий раз делать прогноз с учётом текущего спрогнозируемого символа.
        test_chars = test_chars[1:] + y_pred

    print('')


Итерация 0
1241/1241 [==============================] - 60s 46ms/step - loss: 2.4404
Генерация из посева:	queen said-- get to your place
queen said-- get to your place the said the said the said the said the said the said the said the said the said the said the said 

Итерация 1
1241/1241 [==============================] - 55s 44ms/step - loss: 2.0382
Генерация из посева:	tenberg-tm electronic works ev
tenberg-tm electronic works ever the mast the mast the mast the mast the mast the mast the mast the mast the mast the mast the mas

Итерация 2
1241/1241 [==============================] - 56s 45ms/step - loss: 1.8621
Генерация из посева:	ject gutenberg literary archiv
ject gutenberg literary archive the mare was the said the mare the said the mare the said the mare the said the mare the said the 

Итерация 3
1241/1241 [==============================] - 56s 45ms/step - loss: 1.7436
Генерация из посева:	soon left alone. i wish i hadn
soon left alone. i wish i hadnt to the said the grop on

##### Выводы

Применение различных рекурретных слоёв показывает, что для повышения точности прогноза последовательностей модели необходимо достаточное количество элементов в последовательности.

Так, увеличение последовательности символов с 10 до 30 позволило модели с рекуррентным слоем <code>GRU</code> снизить значение функции потерь кросс-энтропии с 1.3363 до 1.3317.